In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path_data = 'data/Economy_Data.csv'
path_target = 'data/HealthAndPoverty_Data.csv'

X = pd.read_csv(path_data, sep=';')
y = pd.read_csv(path_target, sep=';')

# Limpando dados

In [3]:
X = X[~X['Value'].isna()] # dropping all rows with NaN values
y = y[~y['Value'].isna()] # dropping all rows with NaN values

In [8]:
X['SeriesName']

5        Adjusted net national income (constant 2010 US$)
7        Adjusted net national income (constant 2010 US$)
10             Adjusted net national income (current US$)
11             Adjusted net national income (current US$)
12             Adjusted net national income (current US$)
                               ...                       
86490                Use of IMF credit (DOD, current US$)
86491                Use of IMF credit (DOD, current US$)
86492                Use of IMF credit (DOD, current US$)
86493                Use of IMF credit (DOD, current US$)
86494                Use of IMF credit (DOD, current US$)
Name: SeriesName, Length: 60348, dtype: object

## Primeira ideia:
* Feature selection (ver a coluna Series Name - pra depois agrupamento por ano)


## ATENCAO 
Adicionei abaixo  uma transformacao das features em onehot, mas ainda nao sei como combinar todas no agg (talvez eu mantenha em sum)

In [9]:
one_hot = pd.get_dummies(X['SeriesName'])
X = X.drop('SeriesName', axis=1)
X = X.join(one_hot)

In [14]:
one_hot = pd.get_dummies(y['SeriesName'])
y.drop('SeriesName', axis = 1, inplace=True)
y = y.join(one_hot)

In [10]:
X.head()

,SeriesCode,CountryName,CountryCode,Year,Value,Adjusted net national income (annual % growth),Adjusted net national income (constant 2010 US$),Adjusted net national income (current US$),Adjusted net national income per capita (annual % growth),Adjusted net national income per capita (constant 2010 US$),...,"Total reserves (includes gold, current US$)",Total reserves in months of imports,Total reserves minus gold (current US$),Trade (% of GDP),Trade in services (% of GDP),"Transport services (% of service exports, BoP)","Transport services (% of service imports, BoP)","Travel services (% of service exports, BoP)","Travel services (% of service imports, BoP)","Use of IMF credit (DOD, current US$)"
5,NY.ADJ.NNTY.KD,Brazil,BRA,1970.0,3.918974e+11,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,NY.ADJ.NNTY.KD,India,IND,1970.0,1.915335e+11,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,NY.ADJ.NNTY.CD,Brazil,BRA,1970.0,3.786021e+10,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
11,NY.ADJ.NNTY.CD,China,CHN,1970.0,8.525561e+10,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
12,NY.ADJ.NNTY.CD,India,IND,1970.0,5.776720e+10,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
y.head()

,SeriesCode,CountryName,CountryCode,Year,Value,ARI treatment (% of children under 5 taken to a health provider),"Adolescent fertility rate (births per 1,000 women ages 15-19)",Adults (ages 15+) and children (ages 0-14) newly infected with HIV,Adults (ages 15-49) newly infected with HIV,Age dependency ratio (% of working-age population),...,Tuberculosis treatment success rate (% of new cases),UHC service coverage index,Unmet need for contraception (% of married women ages 15-49),Use of insecticide-treated bed nets (% of under-5 population),Vitamin A supplementation coverage rate (% of children ages 6-59 months),Wanted fertility rate (births per woman),Women who were first married by age 15 (% of women ages 20-24),Women who were first married by age 18 (% of women ages 20-24),Women's share of population ages 15+ living with HIV (%),Young people (ages 15-24) newly infected with HIV
0,SP.ADO.TFRT,Brazil,BRA,1970.0,77.1184,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,SP.ADO.TFRT,China,CHN,1970.0,38.6866,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,SP.ADO.TFRT,India,IND,1970.0,108.3178,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SP.ADO.TFRT,Russian Federation,RUS,1970.0,29.8818,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,SP.ADO.TFRT,South Africa,ZAF,1970.0,93.7106,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


>- Como ideia aqui sugiro contar o número de análises que foram aplicadas aos países. Contar a incidência de dummies para cada ano, e a partir daí ver a evolução das análises empregadas ao longo do tempo.
>- Outra ideia seria montar um dataframe único onde são agregados os dados de `X` e `y`, cruzando indices históricos de cada nação afim de definir possíveis relações entre as variáveis.

In [64]:
X_by_year = X[X['CountryName']=='Brazil'].groupby(['Year']).mean().reset_index()
y_by_year = y[y['CountryName']=='Brazil'].groupby(['Year']).mean().reset_index()

In [66]:
X_by_year.head()

,Year,Value,Adjusted net national income (annual % growth),Adjusted net national income (constant 2010 US$),Adjusted net national income (current US$),Adjusted net national income per capita (annual % growth),Adjusted net national income per capita (constant 2010 US$),Adjusted net national income per capita (current US$),"Adjusted net savings, excluding particulate emission damage (% of GNI)","Adjusted net savings, excluding particulate emission damage (current US$)",...,"Total reserves (includes gold, current US$)",Total reserves in months of imports,Total reserves minus gold (current US$),Trade (% of GDP),Trade in services (% of GDP),"Transport services (% of service exports, BoP)","Transport services (% of service imports, BoP)","Travel services (% of service exports, BoP)","Travel services (% of service imports, BoP)","Use of IMF credit (DOD, current US$)"
0,1970.0,4.388201e+10,0.000000,0.005319,0.005319,0.000000,0.005319,0.005319,0.0,0.0,...,0.005319,0.0,0.005319,0.005319,0.0,0.0,0.0,0.0,0.0,0.005319
1,1971.0,4.801289e+10,0.005181,0.005181,0.005181,0.005181,0.005181,0.005181,0.0,0.0,...,0.005181,0.0,0.005181,0.005181,0.0,0.0,0.0,0.0,0.0,0.005181
2,1972.0,5.340986e+10,0.005102,0.005102,0.005102,0.005102,0.005102,0.005102,0.0,0.0,...,0.005102,0.0,0.005102,0.005102,0.0,0.0,0.0,0.0,0.0,0.005102
3,1973.0,6.158197e+10,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.0,0.0,...,0.005128,0.0,0.005128,0.005128,0.0,0.0,0.0,0.0,0.0,0.005128
4,1974.0,6.799576e+10,0.005102,0.005102,0.005102,0.005102,0.005102,0.005102,0.0,0.0,...,0.005102,0.0,0.005102,0.005102,0.0,0.0,0.0,0.0,0.0,0.005102


In [13]:
y

,SeriesName,SeriesCode,CountryName,CountryCode,Year,Value
0,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,Brazil,BRA,1970.0,77.1184
1,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,China,CHN,1970.0,38.6866
2,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,India,IND,1970.0,108.3178
3,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,Russian Federation,RUS,1970.0,29.8818
4,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,South Africa,ZAF,1970.0,93.7106
...,...,...,...,...,...,...
69322,Probability of dying among youth ages 20-24 ye...,SH.DYN.2024,India,IND,2019.0,6.0000
69323,Probability of dying among youth ages 20-24 ye...,SH.DYN.2024,Russian Federation,RUS,2019.0,4.7000
69324,Probability of dying among youth ages 20-24 ye...,SH.DYN.2024,South Africa,ZAF,2019.0,12.4000
69489,Women's share of population ages 15+ living wi...,SH.DYN.AIDS.FE.ZS,South Africa,ZAF,2019.0,65.2000
